In [2]:
import numpy as np
import pandas as pd 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image
import numpy as np

In [3]:
class ZeroDCE(nn.Module):
    def __init__(self):
        super(ZeroDCE, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1, 1)
        self.conv2 = nn.Conv2d(32, 32, 3, 1, 1)
        self.conv3 = nn.Conv2d(32, 32, 3, 1, 1)
        self.conv4 = nn.Conv2d(32, 3, 3, 1, 1)

    def forward(self, x):
        x1 = F.leaky_relu(self.conv1(x))
        x2 = F.leaky_relu(self.conv2(x1))
        x3 = F.leaky_relu(self.conv3(x2))
        x4 = self.conv4(x3)
        return x + x4

In [4]:
class DnCNN(nn.Module):
    def __init__(self, num_layers=17, num_channels=64):
        super(DnCNN, self).__init__()
        layers = [nn.Conv2d(3, num_channels, kernel_size=3, padding=1), nn.ReLU(inplace=True)]
        for _ in range(num_layers - 2):
            layers.append(nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1))
            layers.append(nn.BatchNorm2d(num_channels))
            layers.append(nn.ReLU(inplace=True))
        layers.append(nn.Conv2d(num_channels, 3, kernel_size=3, padding=1))
        self.dncnn = nn.Sequential(*layers)

    def forward(self, x):
        return x - self.dncnn(x)


In [5]:
class AODNet(nn.Module):
    def __init__(self):
        super(AODNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 3, 1)
        self.conv2 = nn.Conv2d(3, 3, 3, padding=1)
        self.conv3 = nn.Conv2d(6, 3, 5, padding=2)
        self.conv4 = nn.Conv2d(6, 3, 7, padding=3)
        self.conv5 = nn.Conv2d(12, 3, 3, padding=1)

    def forward(self, x):
        x1 = F.relu(self.conv1(x))
        x2 = F.relu(self.conv2(x1))
        cat1 = torch.cat((x1, x2), 1)
        x3 = F.relu(self.conv3(cat1))
        cat2 = torch.cat((x2, x3), 1)
        x4 = F.relu(self.conv4(cat2))
        cat3 = torch.cat((x1, x2, x3, x4), 1)
        k = F.relu(self.conv5(cat3))
        return k * x - k + 1


In [6]:
class DeepRetinex(nn.Module):
    def __init__(self):
        super(DeepRetinex, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 3, 3, padding=1)

    def forward(self, x):
        x1 = F.relu(self.conv1(x))
        x2 = F.relu(self.conv2(x1))
        x3 = self.conv3(x2)
        return torch.sigmoid(x3) * x


In [7]:
import os
import torch
from tqdm import tqdm  # For progress bar


In [8]:

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [9]:
device

device(type='cuda')

In [10]:
# Define the base directory for DAWN dataset
base_dir = '/kaggle/input/dawn-low-visibilty-car-detection/DAWN'
output_dir = '/kaggle/working/preprocessed_images'
os.makedirs(output_dir, exist_ok=True)


In [11]:
# Move the models to GPU
zero_dce = ZeroDCE().to(device)
# dncnn = DnCNN().to(device)
# aod_net = AODNet().to(device)
# deep_retinex = DeepRetinex().to(device)

In [12]:
def preprocess_image(image_path, target_size=(640, 640)):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize(target_size),
        transforms.ToTensor()
    ])
    img_tensor = transform(image).unsqueeze(0).to(device)  # Move tensor to GPU

    # Apply preprocessing techniques
    with torch.no_grad():
        # Low-light enhancement
        enhanced = zero_dce(img_tensor)
        
        # Denoising
#         denoised = dncnn(enhanced)
        
        # Dehazing
#         dehazed = aod_net(denoised)
        
        # Contrast enhancement
#         final = deep_retinex(dehazed)

    # Convert back to CPU and then to PIL Image
    final_image = transforms.ToPILImage()(enhanced.squeeze(0).cpu())  # Move back to CPU
    
    return final_image

In [13]:
# Allowed image extensions
allowed_extensions = ('.jpg', '.jpeg', '.png')

# Iterate through each weather condition folder
for weather_condition in ['Fog', 'Rain', 'Sand', 'Snow']:
    condition_dir = os.path.join(base_dir, weather_condition, weather_condition)
    
    # Create output directory for the current condition
    condition_output_dir = os.path.join(output_dir, weather_condition)
    os.makedirs(condition_output_dir, exist_ok=True)

    # Iterate through all images in the folder
    for img_name in tqdm(os.listdir(condition_dir), desc=f'Processing {weather_condition} images'):
        img_path = os.path.join(condition_dir, img_name)

        # Check if the file has an allowed extension
        if img_name.lower().endswith(allowed_extensions):
            # Preprocess the image using the function you've already written
            processed_image = preprocess_image(img_path)
            
            # Save the preprocessed image
            processed_image.save(os.path.join(condition_output_dir, img_name))

print("All valid images preprocessed and saved on GPU!")

Processing Snow images: 100%|██████████| 206/206 [00:11<00:00, 17.24it/s]

All valid images preprocessed and saved on GPU!


In [14]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 873.6/873.6 kB 6.0 MB/s eta 0:00:0000:0100:01


In [15]:
from ultralytics import YOLO
import os
from PIL import Image
from tqdm import tqdm


In [16]:
model = YOLO('yolov8n.pt').to(device)

100%|██████████| 6.25M/6.25M [00:00<00:00, 77.5MB/s]


In [17]:
# Function to run YOLOv8 detection on an image and return processed image
def detect_and_save_objects(image_path, output_dir):
    # Run detection
    results = model(image_path)
    
    # Render the detection results (annotates the image)
    annotated_img = results[0].plot()  # This creates an annotated numpy array with bounding boxes
    
    # Save the detection image
    img_name = os.path.basename(image_path)  # Keep original image name
    save_path = os.path.join(output_dir, f'detected_{img_name}')
    Image.fromarray(annotated_img).save(save_path)  # Convert numpy array back to image and save



In [18]:
# Directory for detection output
detection_output_dir = '/kaggle/working/detections'
os.makedirs(detection_output_dir, exist_ok=True)


In [19]:
# Run YOLOv8 on preprocessed images
for weather_condition in ['Fog', 'Rain', 'Sand', 'Snow']:
    preprocessed_dir = os.path.join(output_dir, weather_condition)
    
    condition_output_dir = os.path.join(detection_output_dir, weather_condition)
    os.makedirs(condition_output_dir, exist_ok=True)  # Ensure the output directory for each condition exists
    
    # Process each image
    for img_name in tqdm(os.listdir(preprocessed_dir), desc=f'Detecting objects in {weather_condition} images'):
        img_path = os.path.join(preprocessed_dir, img_name)
        
        # Detect and save objects
        detect_and_save_objects(img_path, condition_output_dir)

print("YOLOv8 Detection complete!")


Detecting objects in Fog images:   0%|          | 0/300 [00:00<?, ?it/s]


image 1/1 /kaggle/working/preprocessed_images/Fog/mist-023.jpg: 640x640 11 cars, 3 trucks, 9.1ms
Speed: 5.3ms preprocess, 9.1ms inference, 253.1ms postprocess per image at shape (1, 3, 640, 640)


Detecting objects in Fog images:   0%|          | 1/300 [00:00<04:19,  1.15it/s]


image 1/1 /kaggle/working/preprocessed_images/Fog/mist-009.jpg: 640x640 1 car, 2 trucks, 8.3ms
Speed: 2.1ms preprocess, 8.3ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/working/preprocessed_images/Fog/mist-098.jpg: 640x640 (no detections), 8.0ms
Speed: 2.1ms preprocess, 8.0ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/working/preprocessed_images/Fog/mist-145.jpg: 640x640 (no detections), 8.3ms
Speed: 1.5ms preprocess, 8.3ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/working/preprocessed_images/Fog/mist-034.jpg: 640x640 2 bottles, 8.4ms
Speed: 1.6ms preprocess, 8.4ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/working/preprocessed_images/Fog/foggy-092.jpg: 640x640 (no detections), 8.1ms
Speed: 1.5ms preprocess, 8.1ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)


Detecting objects in Fog images:   2%|▏         | 6/300 [00:00<00:36,  7.98it/s]

...


image 1/1 /kaggle/working/preprocessed_images/Snow/snow_storm-005.jpg: 640x640 1 car, 2 trucks, 1 traffic light, 10.3ms
Speed: 1.6ms preprocess, 10.3ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/working/preprocessed_images/Snow/snow_storm-087.jpg: 640x640 5 cars, 2 traffic lights, 8.4ms
Speed: 1.6ms preprocess, 8.4ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/working/preprocessed_images/Snow/snow_storm-073.jpg: 640x640 3 persons, 10 cars, 4 buss, 1 truck, 8.4ms
Speed: 1.5ms preprocess, 8.4ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/working/preprocessed_images/Snow/snow_storm-239.jpg: 640x640 1 car, 7.9ms
Speed: 2.0ms preprocess, 7.9ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/working/preprocessed_images/Snow/snow_storm-153.jpg: 640x640 1 car, 1 traffic light, 8.4ms
Speed: 1.5ms preprocess, 8.4ms inference, 1.4ms postproc

Detecting objects in Snow images:  98%|█████████▊| 200/204 [00:04<00:00, 43.38it/s]


image 1/1 /kaggle/working/preprocessed_images/Snow/snow_storm-045.jpg: 640x640 2 persons, 2 cars, 8.5ms
Speed: 1.5ms preprocess, 8.5ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/working/preprocessed_images/Snow/snow_storm-293.jpg: 640x640 4 cars, 1 bus, 1 truck, 9.3ms
Speed: 1.5ms preprocess, 9.3ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/working/preprocessed_images/Snow/snow_storm-141.jpg: 640x640 1 person, 3 cars, 1 bus, 1 truck, 8.1ms
Speed: 1.6ms preprocess, 8.1ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/working/preprocessed_images/Snow/snow_storm-370.jpg: 640x640 22 cars, 4 buss, 3 trucks, 8.0ms
Speed: 1.4ms preprocess, 8.0ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)


Detecting objects in Snow images: 100%|██████████| 204/204 [00:04<00:00, 43.80it/s]

YOLOv8 Detection complete!


In [20]:
def estimate_distance(bbox, image_width, focal_length=700):
    car_width_pixels = bbox[2] - bbox[0]  # Width of bounding box in pixels
    real_car_width = 1.8  # Average car width in meters
    distance = (real_car_width * focal_length) / car_width_pixels
    return distance


In [21]:
# Example code to estimate distance for detected objects
for result in results:
    for bbox in result.boxes:  # result.boxes gives you the bounding boxes
        distance = estimate_distance(bbox.xyxy, image_width=result.orig_shape[1])
        print(f"Estimated distance to the car: {distance:.2f} meters")


NameError: name 'results' is not defined

In [ ]:
# Save the YOLO model
model_path = '/kaggle/working/yolov8n_model.pt'
model.save(model_path)


In [ ]:
class CarDetectionPipeline:
    def __init__(self, preprocess, detect, postprocess):
        self.preprocess = preprocess
        self.detect = detect
        self.postprocess = postprocess

    def run_pipeline(self, image_path):
        # Run preprocessing, detection, and post-processing
        preprocessed_image = self.preprocess(image_path)
        detection_results = self.detect(preprocessed_image)
        return self.postprocess(detection_results)

    def postprocess(self, detection_results):
        # Extract the image width from the detection results
        image_width = detection_results[0].orig_shape[1]  # Assuming results contain original image shape
        
        # Call estimate_distance with both detection_results and image_width
        return estimate_distance(detection_results, image_width)


In [ ]:
# Define a regular detection function instead of using lambda
def detect_fn(img):
    return model(img)

# Instantiate the pipeline with the defined functions
pipeline = CarDetectionPipeline(preprocess_image, detect_fn, estimate_distance)

# Now you can use the pipeline object or save it
with open('/kaggle/working/car_detection_pipeline.pkl', 'wb') as f:
    pickle.dump(pipeline, f)

print("Pipeline object created and saved successfully!")


In [ ]:
# Define the actual image path (replace with an actual image file)
image_path = '/kaggle/working/preprocessed_images/Fog/mist-001.jpg'

# Run the pipeline on the actual image
result = pipeline.run_pipeline(image_path)
print(result)


In [ ]:
import shutil

# Define the path to the working directory and output zip file
working_dir = '/kaggle/working'
output_zip = '/kaggle/working/working_directory.zip'

# Zip the working directory
shutil.make_archive(output_zip.replace('.zip', ''), 'zip', working_dir)

print("Working directory zipped successfully!")


In [ ]:
from IPython.display import FileLink

# Create a download link for the zipped working directory
FileLink(output_zip)
